In [19]:
import yaml
import pandas as pd
import os
import datetime as dt
import bokeh.plotting as plotting
from bokeh.io import show
from bokeh.models.glyphs import Circle
from bokeh.models import (
    GMapPlot, GMapOptions, Range1d, ColumnDataSource,
    PanTool, WheelZoomTool, ResetTool, HoverTool,
    ZoomInTool,ZoomOutTool)
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
plotting.output_notebook()

Loading BokehJS ...

In [10]:
# Load Google API credentials
# You will need a file called *credentials.yaml* in your working directory.
# This file needs to be formatted as follows:
# * api_key: 'ABCD999....'
with open('credentials.yaml') as f:
    credentials = yaml.load(f)
API_KEY = credentials['api_key']

In [11]:
def ticks_to_datetime(ticks, rtn_type='datetime'):
    if rtn_type == 'datetime':
        return dt.datetime.fromtimestamp(ticks / 1000)
    elif rtn_type == 'time_string':
        return dt.datetime.fromtimestamp(ticks / 1000).strftime('%H:%M:%S.%f')
    elif rtn_type == 'time_string_short':
        return dt.datetime.fromtimestamp(ticks / 1000).strftime('%H:%M')
    elif rtn_type == 'datetime_string':
        return dt.datetime.fromtimestamp(ticks / 1000).strftime('%m-%d-%Y %H:%M:%S.%f')
def datetime_to_ticks(dt_):
    return dt.datetime.timestamp(dt_)*1000
now=dt.datetime.now()
assert ticks_to_datetime(datetime_to_ticks(now), 'datetime') == now

In [12]:
h5_dir = '/users/lukestarnes/Documents/adsb'
# h5_dir = r'C:\adsb'
h5_files = [os.path.join(h5_dir, f) for f in os.listdir(h5_dir)]
h5_file = '/users/lukestarnes/Documents/adsb/2018-06-11.h5'
with pd.HDFStore(h5_file) as store:
    columns_to_keep = ['Lat','Long','Alt','PosTime','To','From','Icao','Id','Mdl',
                   'Spd', 'Trak', 'TTrk', 'Man', 'Op']
    condition = '(Icao == "896194") | (Icao == "3445D4")'
    df = store.select('data', where=condition,
                           columns=columns_to_keep).dropna(subset=('Lat','Long'))

In [13]:
df['Time'] = df.PosTime.apply(ticks_to_datetime)
df['TimeString'] = df.PosTime.apply(lambda x: ticks_to_datetime(x,"time_string_short")+"Z")

In [15]:
legs = df[['From','To']].drop_duplicates().dropna().set_index('From').to_dict()['To']
for i, (f, t) in enumerate(legs.items()):
    t = [t]
    f = [f]
    criteria = "From == @f & To == @t"
    df.loc[df.eval(criteria), 'Leg'] = i + 1

In [16]:
airport_codes_url='https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
airport_codes = pd.read_csv(airport_codes_url,index_col=0)
airport_codes.columns = ['Name','City','Country','IATA','ICAO','Latitude','Longitude',
                         'Altitude','Timezone','DST','Tz','Type','Source']

In [17]:
legs = df[['From','To']].drop_duplicates().dropna().set_index('From').to_dict()['To']
for i, (f, t) in enumerate(legs.items()):
    print(f'Leg {i+1}:')
    df_leg = df[(df['From'] == f) & (df['To'] == t)]
    from_airport = airport_codes[airport_codes['ICAO'] == f].Name.values[0]
    to_airport = airport_codes[airport_codes['ICAO'] == t].Name.values[0]
    departure_time = ticks_to_datetime(df_leg.PosTime.min(),'time_string_short')
    arrival_time = ticks_to_datetime(df_leg.PosTime.max(),'time_string_short')
    print(f'Departed {from_airport} at {departure_time}Z and arrived at {to_airport} at {arrival_time}Z')

Leg 1:
Departed Adolfo Suárez Madrid–Barajas Airport at 14:13Z and arrived at Brussels Airport at 16:00Z
Leg 2:
Departed Dubai International Airport at 07:15Z and arrived at London Heathrow Airport at 10:55Z
Leg 3:
Departed London Heathrow Airport at 15:11Z and arrived at Dubai International Airport at 18:12Z


In [24]:
x_range = Range1d(-160, 160)
y_range = Range1d(-80, 80)

map_options = GMapOptions(lat=df.Lat.mean(), #lat of center
                          lng=df.Long.mean(), #long of center
                          map_type="roadmap",
                          zoom=4)

plot = GMapPlot(x_range=x_range, y_range=y_range,
                map_options=map_options, api_key=API_KEY)
icoas = ', '.join(list(df.Icao.value_counts().index))
plot.title.text = f"Flights on ICOAs {icoas}"
mapper = linear_cmap(field_name='Leg', palette=Spectral6 ,low=df.Leg.min() ,high=df.Leg.max())
circle = Circle(x="Long", y="Lat", size=3, line_color=None, fill_color=mapper, fill_alpha=0.9)
plot.add_glyph(ColumnDataSource(df), circle)

tooltips = [
    ("ICAO", "@Icao"),
    ("To-From", "@To-@From"),
    ("Lat, Long", "@Lat, @Long"),
    ("Alt", "@Alt'"),
    ('Gnd Speed', '@Spd knts'),
    ('Aircraft', '@Mdl'),
    ('Airline', '@Op'),
    ('Heading','@Trak deg'),
    ('Time', """@TimeString
    <style>
        .bk-tooltip>div:not(:first-child) {display:none;}
        </style>""")]
#https://stackoverflow.com/a/51249293/6334587


plot.add_tools(PanTool(), WheelZoomTool(),ResetTool(),
               ZoomInTool(),ZoomOutTool(),HoverTool(tooltips=tooltips))

show(plot)